# Comparing BBG SWPM Swap on Pricing with `rvcore`

In [1]:
import sys
sys.path.append("../")

import os
from dotenv import dotenv_values
env_path = os.path.join(os.getcwd(), "../.env")
config = dotenv_values(env_path)

In [2]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import pandas as pd
import QuantLib as ql
from datetime import datetime
from typing import List

from core.IRSwaps import IRSwaps, IRSwapQuery, IRSwapValue, IRSwapStructure, IRSwapStructureFunctionMap, IRSwapValueFunctionMap
from core.IRSwaptions import IRSwaptions, IRSwaptionQuery, IRSwaptionValue, IRSwaptionStructure, IRSwaptionStructureFunctionMap, IRSwaptionValueFunctionMap
from core.DataFetching.FixingsFetcher import FixingsFetcher
from core.utils.ql_utils import ql_date_to_datetime, datetime_to_ql_date

import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import matplotlib.dates as mdates
plt.style.use('ggplot')
params = {'legend.fontsize': 'x-large',
        'figure.figsize': (12, 8),
        'axes.labelsize': 'x-large',
        'axes.titlesize':'x-large',
        'xtick.labelsize':'x-large',
        'ytick.labelsize':'x-large'}
pylab.rcParams.update(params)

import nest_asyncio
nest_asyncio.apply()

In [3]:
curve = "USD-SOFR-1D"
swaps_data_source = rf"CSV_{os.path.join(os.getcwd(), "..", "data", "usd_ois.csv")}"

ff = FixingsFetcher(fred_api_key=config["FRED_API_KEY"]) 
fixings = ff.get_fixings("USD-SOFR-1D")

usd_ois = IRSwaps(
    curve=curve,
    data_source=swaps_data_source,
    fixings=fixings,
    ql_interpolation_algo="log_linear",
    # pre_fetch_curves=True,
    error_verbose=True,
    max_njobs=-1,
)

## Pricing Rec 10mm 3Mx10Y @ 3.736141 using 07/22/2024 Curve on Trade Date 07/29/2024

![womp womp](../dump/7-22-2024-swap-swpm.png)

### Check how curve looked:

In [4]:
usd_ois.irswaps_term_structure_plotter(
    bdates=[datetime(2024, 7, 22), datetime(2024, 7, 19), datetime(2024, 7, 15), datetime(2024, 6, 21)], fwd_tenors=["0D", "3M"], use_plotly=True 
)

PRICING IRSWAPS...: 100%|██████████| 4/4 [00:02<00:00,  1.44it/s]


### Check timeseries of spot 10s and 3M Fwd 10s

In [5]:
spot10s_q = IRSwapQuery(tenor="10Y", structure=IRSwapStructure.OUTRIGHT)
fwd3m10s_q = IRSwapQuery(tenor="3Mx10Y", structure=IRSwapStructure.OUTRIGHT)

ts_df = usd_ois.irswaps_timeseries_builder(
    start_date=datetime(2024, 1, 1),
    end_date=datetime(2024, 7, 22),
    queries=[spot10s_q, fwd3m10s_q],
    n_jobs=-1,
)

usd_ois.timeseries_df_plotter(
    df=ts_df, cols_to_plot=[spot10s_q.col_name(), fwd3m10s_q.col_name()], use_plotly=True
)

PRICING IRSWAPS...: 100%|██████████| 278/278 [00:05<00:00, 46.42it/s] 


### Set pricer:

In [4]:
# trade_dt = datetime(2024, 7, 29)

dt = datetime(2025, 5, 5)
ql_curve = usd_ois.fetch_ql_irswap_curves(bdates=[dt], to_pydt=True)[dt]
swap_index = usd_ois.fetch_irswap_index(bdates=[dt], to_pydt=True)[dt]

curve_handle = ql.YieldTermStructureHandle(ql_curve)
ql.Settings.instance().evaluationDate = curve_handle.referenceDate() 

ql_curve, swap_index

BOOTSTRAPPING HISTORICAL IRSWAPS CURVE...: 100%|██████████| 1/1 [00:00<00:00,  6.18it/s]


(<QuantLib.QuantLib.DiscountCurve; proxy of <Swig Object of type 'ext::shared_ptr< InterpolatedDiscountCurve< LogLinear > > *' at 0x0000018EBAB3DF20> >,
 <QuantLib.QuantLib.Sofr; proxy of <Swig Object of type 'ext::shared_ptr< Sofr > *' at 0x0000018EBAB0EFD0> >)

In [11]:
q = IRSwapQuery(
    tenor="10Y",
    structure=IRSwapStructure.OUTRIGHT,
    structure_kwargs={"notional": 100_000_000},
    # trade_date=dt,
    # curve=curve,
    # curve_handle=ql.YieldTermStructureHandle(usd_ois.fetch_ql_irswap_curves(bdates=[trade_dt], to_pydt=True)[trade_dt]),
)
package, risk_weights = IRSwapStructureFunctionMap(curve=curve, curve_handle=curve_handle, swap_index=swap_index).apply(
    structure=q.structure, tenor=q.tenor, effective_date=q.effective_date, maturity_date=q.maturity_date, **q.structure_kwargs
)

swap_value_func_map = IRSwapValueFunctionMap(package=package, risk_weights=risk_weights, curve=curve, curve_handle=curve_handle)

outright: ql.OvernightIndexedSwap = package[0]

print("Eff/Mat Dates ", outright.startDate(), outright.maturityDate())
print("NPV: ", outright.NPV())
print("Par Rate: ", outright.fairRate() * 100)
print("Fixed Rate: ", outright.fixedRate() * 100)
print("Fair Spread (bps): ", outright.fairSpread() * 10_000)
print("Notional: ", outright.nominal())

for irsv in [IRSwapValue.DV01, IRSwapValue.PV01, IRSwapValue.GAMMA_01, IRSwapValue.CARRY_BPS_RUNNING, IRSwapValue.ROLL_BPS_RUNNING]:
    print(irsv.name, swap_value_func_map.apply(irsv, **{"horizon": ql.Period("1Y")}))

Eff/Mat Dates  May 7th, 2025 May 7th, 2035
NPV:  -3.3527612686157227e-08
Par Rate:  3.80878893665942
Fixed Rate:  3.8087889366594156
Fair Spread (bps):  -4.0243121321189305e-13
Notional:  100000000.0
DV01 85922.88720932603
PV01 83312.6546486439
GAMMA_01 81.8971835449338
CARRY_BPS_RUNNING -6.697264724258995
ROLL_BPS_RUNNING 4.198159777678637


In [7]:
datetime_to_ql_date(datetime(2025, 5, 1)) - ql.Period("1M")

Date(1,4,2025)

In [26]:
ql.Settings.instance().evaluationDate = curve_handle.referenceDate() + ql.Period("3M")
outright.fairRate(), ql.Settings.instance().evaluationDate

RuntimeError: 2nd leg: Missing SOFRON Actual/360 fixing for August 30th, 2023